In [1]:

# libraries
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import gc
from matplotlib import style
from pybaseball import statcast
import progressbar 
import warnings
import psycopg2

# pd.options.mode.chained_assignment = None  # default='warn'
# warnings.simplefilter(action='ignore', category=FutureWarning)
# pd.options.display.float_format = '{:.3f}'.format

df_sc = statcast(start_dt='2024-01-01', end_dt='2024-09-30')




This is a large query, it may take a moment to complete
Skipping offseason dates


  0%|          | 0/200 [00:00<?, ?it/s]/opt/homebrew/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
  0%|          | 1/200 [00:08<27:14,  8.21s/it]/opt/homebrew/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/opt/homebrew/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/opt/homebrew/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isi

In [ ]:




df_sc.groupby(['batter', 'events']).launch_speed.agg(['mean', 'count']).reset_index()

,batter,events,mean,count
0,444482,double,95.7,11
1,444482,field_error,97.45,2
2,444482,field_out,86.172897,107
3,444482,fielders_choice,66.9,1
4,444482,fielders_choice_out,97.1,1
...,...,...,...,...
9585,815842,field_out,<NA>,0
9586,815842,strikeout,<NA>,0
9587,815888,field_out,96.8,1
9588,815888,single,92.7,1


In [ ]:
# creating an spray degree column
df_sc['spray_deg'] = np.degrees(np.arctan((df_sc['hc_x'] - 125.42) /  (200 - df_sc['hc_y'])* 0.75 ))

# creating a spray_deg adjusted that accounts for LHH 
df_sc['spray_deg_adj'] = np.where(df_sc['stand'] == 'L', df_sc['spray_deg'] * -1, df_sc['spray_deg'])

df_sc['outcome'] = np.where(df_sc['events'].isin([
    'field_out', 'force_out', 'grounded_into_double_play', 'sac_fly', 'field_error',
    'sac_bunt', 'fielders_choice', 'double_play', 'fielders_choice_out', 
    'sac_fly_double_play', 'triple_play'
]), 'out', df_sc['events'])

df_sc['zone'] = pd.to_numeric(df_sc['zone'], errors='coerce').astype(int)


In [161]:
import pandas as pd
import numpy as np
from scipy.stats import skewnorm
from tqdm import tqdm

df = df_sc.copy()

# Assign velocity bucket
def get_velocity_bucket(v):
    if pd.isna(v):
        return None
    return f"{int(v // 5) * 5}-{int(v // 5) * 5 + 5}"

df['velocity_bucket'] = df['release_speed'].apply(get_velocity_bucket)

# Keep only rows with launch speed and outcome
df = df[df['launch_speed'].notna() & df['outcome'].notna()]

# Fallback context levels
fallback_levels = [
    ['batter', 'outcome', 'pitch_type', 'zone', 'velocity_bucket'],
    ['batter', 'outcome', 'pitch_type', 'zone'],
    ['batter', 'outcome', 'pitch_type'],
    ['batter', 'outcome'],
    ['outcome', 'pitch_type'],
    ['outcome'],
    []
]

ev_fits = {}
completed_keys = set()

outer_group_cols = ['batter', 'game_year', 'stand', 'p_throws']
outer_grouped = df.groupby(outer_group_cols)

for outer_key, outer_df in tqdm(outer_grouped, desc="Top-level groups"):
    outer_key = (outer_key,) if not isinstance(outer_key, tuple) else outer_key

    for level_keys in fallback_levels:
        if level_keys:
            grouped = outer_df.groupby(level_keys)
            tqdm_bar = tqdm(grouped, total=len(grouped), desc=f"Fitting: {level_keys}", leave=False)

            for key, group in tqdm_bar:
                key = (key,) if not isinstance(key, tuple) else key
                full_key_id = (outer_key, tuple(level_keys), key)

                if full_key_id in completed_keys:
                    continue

                evs = pd.to_numeric(group['launch_speed'], errors='coerce').dropna()
                evs = evs[evs.apply(lambda x: isinstance(x, (int, float)) and np.isfinite(x))]

                if len(evs) < 10:
                    continue

                try:
                    a, loc, scale = skewnorm.fit(evs.to_numpy(dtype=np.float64))
                    ev_fits[full_key_id] = {
                        'batter': outer_key[0],
                        'game_year': outer_key[1],
                        'stand': outer_key[2],
                        'p_throws': outer_key[3],
                        'keys': level_keys,
                        'key_values': key,
                        'skew': a,
                        'mean': loc,
                        'std': scale,
                        'n': len(evs)
                    }
                    completed_keys.add(full_key_id)
                except Exception as e:
                    print(f"Skipping {key} at level {level_keys} for group {outer_key} due to error: {e}")
        else:
            # Global fallback for this outer group
            evs = pd.to_numeric(outer_df['launch_speed'], errors='coerce').dropna()
            evs = evs[evs.apply(lambda x: isinstance(x, (int, float)) and np.isfinite(x))]

            if len(evs) >= 10:
                try:
                    a, loc, scale = skewnorm.fit(evs.to_numpy(dtype=np.float64))
                    ev_fits[(outer_key, (), ())] = {
                        'batter': outer_key[0],
                        'game_year': outer_key[1],
                        'stand': outer_key[2],
                        'p_throws': outer_key[3],
                        'keys': [],
                        'key_values': (),
                        'skew': a,
                        'mean': loc,
                        'std': scale,
                        'n': len(evs)
                    }
                except Exception as e:
                    print(f"Skipping global fallback for {outer_key} due to error: {e}")


Top-level groups: 100%|██████████| 1731/1731 [08:34<00:00,  3.36it/s]


In [168]:

import pandas as pd
import numpy as np
from tqdm import tqdm

df = df_sc.copy()

# Assign velocity bucket
def get_velocity_bucket(v):
    if pd.isna(v):
        return None
    return f"{int(v // 5) * 5}-{int(v // 5) * 5 + 5}"

df['velocity_bucket'] = df['release_speed'].apply(get_velocity_bucket)

# Keep only rows with launch speed and outcome
df = df[df['launch_speed'].notna() & df['outcome'].notna()]

# Fallback context levels
fallback_levels = [
    ['outcome', 'pitch_type', 'zone', 'velocity_bucket'],
    ['outcome', 'pitch_type', 'zone'],
    ['outcome', 'pitch_type'],
    ['outcome'],
    []
]

ev_fits = {}
completed_keys = set()

outer_group_cols = ['batter', 'game_year', 'stand', 'p_throws']
outer_grouped = df.groupby(outer_group_cols)

for outer_key, outer_df in tqdm(outer_grouped, desc="Top-level groups"):
    outer_key = (outer_key,) if not isinstance(outer_key, tuple) else outer_key

    for level_keys in fallback_levels:
        if level_keys:
            grouped = outer_df.groupby(level_keys)
            tqdm_bar = tqdm(grouped, total=len(grouped), desc=f"Fitting: {level_keys}", leave=False)

            for key, group in tqdm_bar:
                key = (key,) if not isinstance(key, tuple) else key
                full_key_id = (outer_key, tuple(level_keys), key)

                if full_key_id in completed_keys:
                    continue

                evs = pd.to_numeric(group['launch_speed'], errors='coerce').dropna()
                evs = evs[evs.apply(lambda x: isinstance(x, (int, float)) and np.isfinite(x))]

                if len(evs) < 5:
                    continue

                # ✅ Empirical stats only
                a = -0.8
                loc = evs.mean()
                scale = evs.std()

                ev_fits[full_key_id] = {
                    'batter': outer_key[0],
                    'game_year': outer_key[1],
                    'stand': outer_key[2],
                    'p_throws': outer_key[3],
                    'keys': level_keys,
                    'key_values': key,
                    'skew': a,   # always 0
                    'mean': loc,
                    'std': scale,
                    'n': len(evs)
                }
                completed_keys.add(full_key_id)
        else:
            # Global fallback for this outer group
            evs = pd.to_numeric(outer_df['launch_speed'], errors='coerce').dropna()
            evs = evs[evs.apply(lambda x: isinstance(x, (int, float)) and np.isfinite(x))]

            if len(evs) >= 5:
                a = -0.8
                loc = evs.mean()
                scale = evs.std()

                ev_fits[(outer_key, (), ())] = {
                    'batter': outer_key[0],
                    'game_year': outer_key[1],
                    'stand': outer_key[2],
                    'p_throws': outer_key[3],
                    'keys': [],
                    'key_values': (),
                    'skew': a,
                    'mean': loc,
                    'std': scale,
                    'n': len(evs)
                }




Top-level groups: 100%|██████████| 1731/1731 [02:26<00:00, 11.85it/s]


In [169]:
flattened = []

for _, fit in ev_fits.items():
    row = {
        'batter': fit['batter'],
        'game_year': fit['game_year'],
        'stand': fit['stand'],
        'p_throws': fit['p_throws'],
        'skew': fit['skew'],
        'mean': fit['mean'],
        'std': fit['std'],
        'n': fit['n'],
        'level': "_".join(fit['keys']) if fit['keys'] else 'global'
    }

    # Guarantee all context fields
    context_keys = ['outcome', 'pitch_type', 'zone', 'velocity_bucket']
    context_map = dict(zip(fit['keys'], fit['key_values']))

    for key in context_keys:
        val = context_map.get(key, pd.NA)
        if key == 'zone':
            try:
                val = int(val) if pd.notna(val) and float(val).is_integer() else pd.NA
            except:
                val = pd.NA
        row[key] = val

    flattened.append(row)

# Finalize dataframe
dist_df = pd.DataFrame(flattened)

# Ensure correct column order
expected_columns = [
    'batter', 'game_year', 'stand', 'p_throws',
    'outcome', 'pitch_type', 'zone', 'velocity_bucket',
    'skew', 'mean', 'std', 'n', 'level'
]
dist_df = dist_df[expected_columns]

# Clean nulls
dist_df['zone'] = pd.to_numeric(dist_df['zone'], errors='coerce').astype('Int64')
object_cols = dist_df.select_dtypes(include='object').columns
dist_df[object_cols] = dist_df[object_cols].fillna('')

# Export CSV
dist_df.to_csv(
    "/Users/loganmottley/Desktop/Projects/go-baseball/data/ev_distributions_by_batter_year_handedness.csv",
    index=False,
    na_rep=''
)


In [167]:

dist_df


,batter,game_year,stand,p_throws,outcome,pitch_type,zone,velocity_bucket,skew,mean,std,n,level
0,444482,2024,L,L,out,,<NA>,,-0.8,78.587500,16.043996,8,batter_outcome
1,444482,2024,L,L,out,,<NA>,,-0.8,78.587500,16.043996,8,outcome
2,444482,2024,L,L,,,<NA>,,-0.8,79.266667,20.756613,12,global
3,444482,2024,L,R,out,FF,6,95-100,-0.8,87.380000,22.390891,5,batter_outcome_pitch_type_zone_velocity_bucket
4,444482,2024,L,R,out,CH,4,,-0.8,83.942857,15.686497,7,batter_outcome_pitch_type_zone
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27276,808982,2024,L,R,out,SL,<NA>,,-0.8,88.460000,9.108994,10,outcome_pitch_type
27277,808982,2024,L,R,single,FF,<NA>,,-0.8,90.900000,10.763944,9,outcome_pitch_type
27278,808982,2024,L,R,out,,<NA>,,-0.8,89.389855,11.477204,69,outcome
27279,808982,2024,L,R,single,,<NA>,,-0.8,87.843478,17.875194,23,outcome


In [159]:

dist_df.dtypes


batter               int64
game_year            int64
stand               object
p_throws            object
skew               float64
mean               float64
std                float64
n                    int64
level               object
zone                 Int64
outcome             object
pitch_type          object
velocity_bucket     object
dtype: object